In [1]:
#|hide
#| eval: false
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [1]:
#|default_exp callback.galileo

In [3]:
#|export
from __future__ import annotations

import pandas as pd

from functools import partial

from torch.nn import Module
from typing import Any, Callable, Dict, List

from fastai.learner import Callback
from fastai.data.load import DataLoader

In [4]:
#|hide
from nbdev.showdoc import *

# Galileo

> Integration with [Galileo](https://docs.rungalileo.io/integrations/fastai) 

To use Galileo on your dataset need to install the dataquality client and register your account.
```
pip install dataquality
```


1. Create account: [rungalileo.io](https://console.cloud.rungalileo.io/sign-up).
2. Grab your [token](https://console.cloud.rungalileo.io/get-token). 



In [5]:
#|export
import dataquality

[02/09/23 22:28:23] ERROR    issue loading plot                                                     ]8;id=57450;file:///Users/Shared/anaconda3/lib/python3.9/site-packages/vaex/__init__.py\__init__.py]8;;\:]8;id=986388;file:///Users/Shared/anaconda3/lib/python3.9/site-packages/vaex/__init__.py#805\805]8;;\
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/Users/Shared/anaconda3/lib/python3.9/site-packages/vaex/__init__.py"                
                             , line 802, in <module>                                                               
                                 add_namespace = entry.load()                                                      
                               File                                                                                
                             "/Users/Shared/anaconda3/lib/python3.9/site-packages/pkg_resources/__i                
                             nit__.py", line 2471, in load                                                         
                                 return self.resolve()                                                             
                               File                                                                                
                             "/Users/Shared/anaconda3/lib/python3.9/site-packages/pkg_resources/__i                
                             nit__.py", line 2477, in resolve                                                      
                                 module = __import__(self.module_name, fromlist=['__name__'],                      
                             level=0)                                                                              
                             ModuleNotFoundError: No module named 'vaex.viz'                                       

                    ERROR    issue loading astro                                                    ]8;id=127024;file:///Users/Shared/anaconda3/lib/python3.9/site-packages/vaex/__init__.py\__init__.py]8;;\:]8;id=406842;file:///Users/Shared/anaconda3/lib/python3.9/site-packages/vaex/__init__.py#805\805]8;;\
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/Users/Shared/anaconda3/lib/python3.9/site-packages/vaex/__init__.py"                
                             , line 802, in <module>                                                               
                                 add_namespace = entry.load()                                                      
                               File                                                                                
                             "/Users/Shared/anaconda3/lib/python3.9/site-packages/pkg_resources/__i                
                             nit__.py", line 2471, in load                                                         
                                 return self.resolve()                                                             
                               File                                                                                
                             "/Users/Shared/anaconda3/lib/python3.9/site-packages/pkg_resources/__i                
                             nit__.py", line 2477, in resolve                                                      
                                 module = __import__(self.module_name, fromlist=['__name__'],                      
                             level=0)                                                                              
                             ModuleNotFoundError: No module named 'vaex.astro'                                     

In [6]:
#|export

class _PatchDataLoaderGetIdxs:
    """
    Patch the DataLoader to store the indices of the batches.
    For example:
    self.dl.get_idxs = _PatchDataLoaderGetIdxs(self.dl.get_idxs, self.idx_log)
    """

    def __init__(self, old_func: Callable, store: Dict[str, Any]):
        """
        Patch the DataLoader to store the indices of the batches.
        For example:
        self.dl.get_idxs = IdxLogPatch(self.dl.get_idxs, self.idx_log)
        :param old_func: The original function to patch.
        :param store: The store to store the indices in.
        """
        self.old_func = old_func
        self.store = store
        self.store["dataloader_indices"] = []

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        """
        Call the original function and store the indices.
        :param args: The arguments to pass to the original function.
        :param kwargs: The keyword arguments to pass to the original function.

        """
        res = self.old_func(*args, **kwargs)
        if res:
            self.store["dataloader_indices"].append(res)
        return res

In [7]:
#|export
class DQCallback(Callback):
    """
    Saves training embeddings + logits to find mislabelled samples.
    """

    
    hook = None
    is_initialized = False
    labels = None

    def __init__(self, labels: List[str],
                 layer: Any=None,
                 log_dataset:bool=True,
                 options:Dict[str,Any]={},
                 *args: Any, **kwargs: Any) -> None:
        """
        Callback to log the model training for data quality.
        :param layer: Classifier layer with embeddings as input and logits as output.
        :param log_dataset: Enable automatic extraction of the dataset to data quality.
        :param disable_dq: Disable data quality logging.
        :param args: The arguments to pass to the super class.
        :param kwargs: The keyword arguments to pass to the super class.
        """
        super().__init__(*args, **kwargs)
        self.disable_dq = os.environ.get("DQ_NOOP",False)
        self.labels = labels
        self.log_dataset = log_dataset
        self.options=options
        self.layer = layer
        self.model_outputs_log = {}
        self.current_idx = []
        self.idx_store = {"idx_queue": []}

        assert (
            self.labels is not None
        ), "Labels must be provided. DQFastAiCallback(labels=['negative','positive'])"

        if not self.disable_dq:
            dataquality.init(**options)

    def get_layer(self) -> Module:
        """
        Get the classifier layer, which inputs and outputs will be logged
        (embeddings and logits).
        :return: The classifier layer.
        """
        if self.layer is None:
            # Get the last layer of the model
            return list(list(self.model.children())[-1].children())[-1]
        else:
            return self.layer

    def before_epoch(self) -> None:
        """
        Sets the epoch in data quality.
        """
        if not self.disable_dq:
            dataquality.set_epoch(self.epoch)

    def before_fit(self) -> None:
        if self.is_initialized or self.disable_dq:
            return
        self.wrap_indices()
        self.register_hooks()
        self.log_data()
        self.is_initialized = True

    def before_train(self) -> None:
        """
        Sets the split in data quality and registers the classifier layer hook.
        """
        if not self.disable_dq:
            dataquality.set_split(dataquality.schemas.split.Split.train)
            self.wrap_indices()
            if self.is_initialized:
                return
            self.register_hooks()
            self.log_data()
            self.is_initialized = True

    def wrap_indices(self) -> None:
        """
        Wraps the get_idxs function of the dataloader to store the indices.
        """
        if not hasattr(self,"dl"):
          return
        if not isinstance(self.dl.get_idxs, _PatchDataLoaderGetIdxs):
            self.dl.get_idxs = _PatchDataLoaderGetIdxs(self.dl.get_idxs, self.idx_store)

    def before_validate(self):
        """
        Sets the split in data quality and registers the classifier layer hook.
        """
        if not self.disable_dq:
            dataquality.set_split(dataquality.schemas.split.Split.validation)
        self.wrap_indices()

    def log_data(self):
        """
        Uploads data to galileo and removes the classifier layer hook.
        """
        if self.disable_dq or not self.log_dataset: return
            
        dataquality.set_labels_for_run(self.labels)
        
        train_dl, valid_dl = self.dls
        dataquality.log_image_dataset(  # TODO: add support for other datasets
                self.convert_dl_to_df(train_dl),
                imgs_colname="image",
                imgs_location_colname="path",
                split=dataquality.schemas.split.Split.train,
            )
        dataquality.log_image_dataset(
                self.convert_dl_to_df(valid_dl),
                imgs_colname="image",
                imgs_location_colname="path",
                split=dataquality.schemas.split.Split.validation,
            )

    def after_fit(self):
        """
        Uploads data to galileo and removes the classifier layer hook.
        """
        if self.disable_dq: return

        dataquality.finish()
        try:
            self.h.remove()
        except Exception:
            pass

    def before_batch(self):
        """
        Clears the model outputs log.
        """
        self.model_outputs_log.clear()

    def after_batch(self) -> None:
        """
        Logs the model outputs.
        """
        # If the current batch is empty, get the next batch ids from the store
        if len(self.idx_store["idx_queue"]) == 0  and \
          len(self.idx_store["dataloader_indices"]):
            indices = self.idx_store["dataloader_indices"].pop(0)
            assert isinstance(indices,list) 
            self.idx_store["idx_queue"] = indices
        # Get the current batch size
        bs_len = len(self.model_outputs_log["model_output"])
        # Store the current batch ids by trimming the stored ids by
        # the batch size length
        self.model_outputs_log["ids"] = self.idx_store[
            "idx_queue"
        ][:bs_len]
        self.idx_store["idx_queue"] = self.idx_store[
            "idx_queue"][bs_len:]
        # Log the model outputs
        embs = self.model_outputs_log["model_input"][0].detach().cpu().numpy()
        logits = self.model_outputs_log["model_output"].detach().cpu().numpy()
        ids = self.model_outputs_log["ids"]
        equal_len = len(embs) == len(logits) and len(ids) == len(embs)
        if not equal_len:
            print(f"length not equal. logits: {len(logits)},ids: {len(ids)},\
embs: {len(embs)}")
        if self.disable_dq or not equal_len: return
        dataquality.log_model_outputs(embs=embs, logits=logits, ids=ids)

    def register_hooks(self) -> None:
        """
        Registers the classifier layer hook.
        """
        h = None
        if not self.hook:
            forward_hook = partial(self.forward_hook_with_store, self.model_outputs_log)
            h = self.get_layer().register_forward_hook(forward_hook)
            self.hook = h
        return h

    def forward_hook_with_store(
            self,
        store: Dict[str, Any], layer: Module, model_input: Any, model_output: Any
    ) -> None:
        """
        Forward hook to store the output of a layer.
        :param store: Dictionary to store the output in.
        :param layer: Layer to store the output of.
        :param model_input: Input to the model.
        :param model_output: Output of the model.
        :return: None
        """
        store["model_input"] = model_input
        store["model_output"] = model_output

        
    def convert_dl_to_df(self, dl: DataLoader, x_col: str = "image") -> pd.DataFrame:
        """
        Converts a fastai DataLoader to a pandas DataFrame.
        :param dl: Fast ai DataLoader to convert.
        :param x_col: Name of the column to use for the x values, for example text or image.
        :return: Pandas DataFrame with the data from the DataLoader.
        """
        import vaex
        additonal_data = {}
        if x_col == "image":
            additonal_data["path"] = dl.items
        x, y = [], []
        for x_item, y_item in dl.dataset:
            x.append(x_item)
            y.append(int(y_item))
        return pd.DataFrame(
            {"id": range(dl.n), x_col: x, "label": y, **additonal_data}
            )

In [9]:
show_doc(DQCallback)

---

### DQCallback

>      DQCallback (labels:List[str], layer:Any=None, log_dataset:bool=True,
>                  options:Dict[str,Any]={}, *args:Any, **kwargs:Any)

Saves training embeddings + logits to find mislabelled samples.

In [10]:
#|export
_all_ = []

## Export -

In [11]:
#|hide
from nbdev import *
nbdev_export()